In [1]:
import os
# set device GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
import time
from tensorflow import keras
from tensorflow.keras import layers

tf.enable_eager_execution()
tf.test.is_gpu_available()

import sys
sys.path.append('../')

from config import *

from tools import *


import warnings

warnings.filterwarnings('ignore')
tf.debugging.set_log_device_placement(True)

In [4]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
COLS_NAME = ['ad_id', 'product_id', 'advertiser_id']
BATCH_SIZE = 256
BUFFER_SIZE = 1024

### 载入数据

In [5]:
# tr_user_log = pd.read_pickle(TRAIN_DIR+USER_LOG_PATH)
# ts_user_log = pd.read_pickle(TEST_DIR+USER_LOG_PATH)

# tr_ad_id_log = pd.read_pickle(TRAIN_DIR+AD_INFO_PATH)
# ts_ad_id_log = pd.read_pickle(TEST_DIR+AD_INFO_PATH)

# tr_df = pd.concat([tr_user_log, tr_ad_id_log], axis=1)
# ts_df = pd.concat([ts_user_log, ts_ad_id_log], axis=1)

# tr_df = tr_df[['user_id', 'age', 'gender', 'ad_id', 'product_id', 'advertiser_id', 'click_times']]
# ts_df = ts_df[['user_id', 'ad_id', 'product_id', 'advertiser_id', 'click_times']]

# tr_df['age'] = tr_df['age'] - 1
# tr_df['gender'] = tr_df['gender'] - 1

### 载入gensim预训练词典

In [6]:
vocab_sizes = []
wv_matrixes = []
for col in COLS_NAME:
    #load w2v matrix
    f = open(TRAIN_DIR+'gensim_%s_dict.js'%col,'r')
    a = f.read()
    vocab_dict = eval(a)
    f.close()
    filter_keys = set(vocab_dict.keys())
    vocab_size = len(filter_keys) + 1

    wv_matrix = np.load(TRAIN_DIR+'gensim_%s.npy'%col)
    row = np.random.uniform(size=(1, wv_matrix.shape[1]))

    wv_matrix = np.concatenate([row, wv_matrix], axis=0)

#     ### process and get click list
#     tr_df[col] = tr_df[col].astype(str)
#     ts_df[col] = ts_df[col].astype(str)

#     tr_df = tr_df[tr_df[col].isin(filter_keys)]
#     ts_df = ts_df[ts_df[col].isin(filter_keys)]

#     tr_df[col] = tr_df[col].map(lambda x: vocab_dict[x]) + 1
#     ts_df[col] = ts_df[col].map(lambda x: vocab_dict[x]) + 1
    
    vocab_sizes.append(vocab_size)
    wv_matrixes.append(wv_matrix)

### 提取点击列表和对应的广告信息

In [7]:
# def get_clk_list(df):
#     return list(df.values)

# tr_df = tr_df.groupby(['user_id']).agg({'age': 'first', 
#                                         'gender': 'first', 
#                                         'click_times': lambda x: get_clk_list(x), 
#                                         'ad_id': lambda x: get_clk_list(x),
#                                         'product_id': lambda x: get_clk_list(x),
#                                         'advertiser_id': lambda x: get_clk_list(x)}).reset_index()

# ts_df = ts_df.groupby(['user_id']).agg({'click_times': lambda x: get_clk_list(x), 
#                                         'ad_id': lambda x: get_clk_list(x),
#                                         'product_id': lambda x: get_clk_list(x),
#                                         'advertiser_id': lambda x: get_clk_list(x)}).reset_index()

# tr_df.to_pickle('/home/baode/huangzc/tencent/data/train_preliminary/3shuru.pkl')

# ts_df.to_pickle('/home/baode/huangzc/tencent/data/test/3shuru.pkl')

In [8]:
tr_df = pd.read_pickle('/home/baode/huangzc/tencent/data/train_preliminary/3shuru.pkl')
ts_df = pd.read_pickle('/home/baode/huangzc/tencent/data/test/3shuru.pkl')

### 切分 train 和 test 数据集

In [9]:
msk = np.random.rand(len(tr_df)) <= 0.8
vl_df = tr_df[~msk]
tr_df = tr_df[msk]

### 配置超参数

In [28]:
d_model = 128

EPOCHS = 1

### padding 和mask

In [32]:
### cut lenght and pad sequence
sentence_size = int(min(tr_df['ad_id'].map(lambda x: len(x)).quantile(0.90), ts_df['ad_id'].map(lambda x: len(x)).quantile(0.99)))
print('max len: ', sentence_size)

### pad or trunc
def pad_or_trunc(t):
    dim = tf.size(t)
    return tf.cond(tf.equal(dim, sentence_size), lambda: t,
                    lambda: tf.cond(tf.greater(dim, sentence_size), lambda: tf.slice(t, [0], [sentence_size]), 
                                     lambda: tf.concat([t, tf.zeros(dtype=tf.int64, shape=sentence_size-dim)], 0)))

max len:  65


### 生成dataset

In [34]:
### make train dataset
def gen():
    for row in tr_df.itertuples():
        ad_id_li, product_id_li, advertiser_id_li, age, gender = getattr(row, COLS_NAME[0]),\
                                                                         getattr(row, COLS_NAME[1]),\
                                                                         getattr(row, COLS_NAME[2]),\
                                                                         getattr(row, 'age'), \
                                                                         getattr(row, 'gender')

        yield (ad_id_li, product_id_li, advertiser_id_li, (age, gender))

tr_ds = tf.data.Dataset.from_generator(
     gen,
     (tf.int64, tf.int64, tf.int64, (tf.int64, tf.int64)), 
     (tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([None]), (tf.TensorShape([]), tf.TensorShape([]))))

Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [35]:
### make valid dataset
def gen():
    for row in vl_df.itertuples():
        ad_id_li, product_id_li, advertiser_id_li, age, gender = getattr(row, COLS_NAME[0]),\
                                                                         getattr(row, COLS_NAME[1]),\
                                                                         getattr(row, COLS_NAME[2]),\
                                                                         getattr(row, 'age'), \
                                                                         getattr(row, 'gender')

        yield (ad_id_li, product_id_li, advertiser_id_li, (age, gender))

vl_ds = tf.data.Dataset.from_generator(
     gen,
     (tf.int64, tf.int64, tf.int64, (tf.int64, tf.int64)), 
     (tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([None]), (tf.TensorShape([]), tf.TensorShape([]))))

Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [36]:
### make test dataset
def gen():
    for row in ts_df.itertuples():
        ad_id_li, product_id_li, advertiser_id_li = getattr(row, COLS_NAME[0]),\
                                                             getattr(row, COLS_NAME[1]),\
                                                             getattr(row, COLS_NAME[2])
        yield (ad_id_li, product_id_li, advertiser_id_li)

ts_ds = tf.data.Dataset.from_generator(
     gen,
     (tf.int64, tf.int64, tf.int64), 
     (tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([None])))

Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [37]:
tr_ds = tr_ds.map(lambda ad, product, advertiser, pair: (pad_or_trunc(ad), 
                                                          pad_or_trunc(product), 
                                                          pad_or_trunc(advertiser), 
                                                          pair), num_parallel_calls=tf.data.experimental.AUTOTUNE)

tr_ds = tr_ds.cache()
tr_ds = tr_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
tr_ds = tr_ds.prefetch(tf.data.experimental.AUTOTUNE)

vl_ds = vl_ds.map(lambda ad, product, advertiser, pair: (pad_or_trunc(ad), 
                                                          pad_or_trunc(product), 
                                                          pad_or_trunc(advertiser), 
                                                          pair), num_parallel_calls=tf.data.experimental.AUTOTUNE)

vl_ds = vl_ds.cache()
vl_ds = vl_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
vl_ds = vl_ds.prefetch(tf.data.experimental.AUTOTUNE)

ts_ds = ts_ds.map(lambda ad, product, advertiser: (pad_or_trunc(ad), 
                                                    pad_or_trunc(product), 
                                                    pad_or_trunc(advertiser)), num_parallel_calls=tf.data.experimental.AUTOTUNE)

Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [38]:
tf.reset_default_graph()

### 优化器和学习率

In [39]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [40]:
# with tf.device('/gpu:6'):
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:CPU:0


### 创建模型

In [41]:
class MyModel(tf.keras.Model):
    def __init__(self, d_model, input_vocab_size, 
                 weights=None,
                 lstm_dims=32,
                 dim1=64, dim2=32, num_class1=10, num_class2=2):
        '''
         d_model: dimension of embedding;
         input_vocab_size: vocab size;
         rate: dropout rate;
         weights: pre-trained embedding weights;
        '''
        super(MyModel, self).__init__()
        
        self.lstms = []
        self.embs = []
        if weights is  not None:        
            for i, _ in enumerate(COLS_NAME):
                self.embs.append(tf.keras.layers.Embedding(input_vocab_size[i], 
                                                           d_model, 
                                                           embeddings_initializer=tf.keras.initializers.Constant(weights[i]),
                                                           trainable=False))
                self.lstms.append(tf.keras.layers.LSTM(lstm_dims))
        else:
            for i, _ in enumerate(COLS_NAME):
                self.embs.append(tf.keras.layers.Embedding(input_vocab_size[i], 
                                                           d_model))                
                self.lstms.append(tf.keras.layers.LSTM(lstm_dims))         
        
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.dense1 = tf.keras.layers.Dense(dim1, activation='relu', name='dense1')
        self.dense2 = tf.keras.layers.Dense(dim2, activation='relu', name='dense2')
        self.dense3_age = tf.keras.layers.Dense(num_class1, activation='softmax', name='softmax1')
        self.dense_gender = tf.keras.layers.Dense(num_class2, activation='softmax', name='softmax2')

    def call(self, x):
        res = []
        for i, (emb, lstm) in enumerate(zip(self.embs, self.lstms)):
            e = emb(x[i])
            res.append(lstm(e))

        x = self.concat(res)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3_age(x), self.dense_gender(x)

In [42]:
model = MyModel(d_model=d_model,  
                input_vocab_size=vocab_sizes,
                weights=wv_matrixes)

In [23]:
### test
# model((tf.random.uniform((1, 60)), 
#        tf.random.uniform((1, 60)), 
#        tf.random.uniform((1, 60)), 
#        tf.random.uniform((1, 60))), 
#       training=False,
#       mask=tf.random.uniform((1, 1, 1, 60)),       
#       attention_mask=tf.random.uniform((1, 1, 1, 60)),
#       pool_mask=tf.random.uniform((1,60,1)))

### 损失函数和metric

In [43]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

def loss_function(real, pred):
    pred_age = pred[0]
    real_age = real[0]

    pred_gender = pred[1]
    real_gender = real[1]

    loss1 = loss_object(real_age, pred_age)
    loss2 = loss_object(real_gender, pred_gender)

    loss_ = 0.5*loss1 + 0.5*loss2
    return tf.reduce_mean(loss_)

### train metric
train_loss = tf.keras.metrics.Mean(name='train_loss')
age_train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
gender_train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

### test metric
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
age_valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')
gender_valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

### 开始训练

In [44]:
@tf.function
def train_step(inp, tar):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = loss_function(tar, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    age_train_accuracy(tar[0], predictions[0])
    gender_train_accuracy(tar[1], predictions[1])

@tf.function
def valid_step(inp, tar):
    predictions = model(inp)
    loss = loss_function(tar, predictions)

    valid_loss(loss)
    age_valid_accuracy(tar[0], predictions[0])
    gender_valid_accuracy(tar[1], predictions[1])

In [46]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    age_train_accuracy.reset_states()
    gender_train_accuracy.reset_states()

    '''
    inp1: ad_id list;
    inp2: product_id list;
    inp3: advertiser_id list;
    inp4: click_times list;
    '''
    for (batch, (inp1, inp2, inp3, tar)) in enumerate(tr_ds):
        train_step((inp1, inp2, inp3), tar)

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Age-Accuracy {:.4f} Gender-Accuracy {:.4f} Time taken for training: {} secs'
                   .format(epoch + 1, batch, train_loss.result(), 
                           age_train_accuracy.result(), gender_train_accuracy.result(),
                          time.time()-start))
            start = time.time()
            
    tmp = time.time()
    for inp1, inp2, inp3, tar in vl_ds:
        valid_step((inp1, inp2, inp3), tar)    
    print('########################################## valid ################################################')
    print('Epoch {} Batch {} Loss {:.4f} Age-Accuracy {:.4f} Gender-Accuracy {:.4f} \
          Time taken for validation: {} secs'
          .format(epoch + 1, batch, 
                  valid_loss.result(), age_valid_accuracy.result(), gender_valid_accuracy.result(),
                  time.time() - tmp))

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:

Epoch 1 Batch 2800 Loss 1.0996 Age-Accuracy 0.2783 Gender-Accuracy 0.8280 Time taken for training: 28.582879781723022 secs
Executing op __inference_train_step_41115 in device /job:localhost/replica:0/task:0/device:GPU:0


In [47]:
tmp = time.time()
for inp1, inp2, inp3, tar in vl_ds:
    valid_step((inp1, inp2, inp3), tar)    
print('########################################## valid ################################################')
print('Epoch {} Batch {} Loss {:.4f} Age-Accuracy {:.4f} Gender-Accuracy {:.4f} \
      Time taken for validation: {} secs'
      .format(epoch + 1, batch, 
              valid_loss.result(), age_valid_accuracy.result(), gender_valid_accuracy.result(),
              time.time() - tmp))

Executing op __inference_valid_step_42089 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_valid_step_46562 in device /job:localhost/replica:0/task:0/device:GPU:0
########################################## valid ################################################
Epoch 1 Batch 2812 Loss 0.8766 Age-Accuracy 0.3721 Gender-Accuracy 0.9217       Time taken for validation: 59.73349046707153 secs


Executing op __inference_valid_step_42089 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_valid_step_46562 in device /job:localhost/replica:0/task:0/device:GPU:0
########################################## valid ################################################
Epoch 1 Batch 2812 Loss 0.8766 Age-Accuracy 0.3721 Gender-Accuracy 0.9217       Time taken for validation: 59.73349046707153 secs